# Install and Imports

In [1]:
!pip install ortools

In [4]:
from random import randint, uniform
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
from time import time

# Data Generation

In [7]:
def generate_tech_matrix(cant_variables, cant_disponibilidad, min_requerimientos = 0.0, max_requerimientos = 1.0, rand_type = 'float'):
    matrix = []
    for i in range(cant_variables):
        if rand_type == 'float':
            req = [ round(uniform(min_requerimientos, max_requerimientos), 2) for _ in range(cant_disponibilidad) ]
        else:
            req = [ randint(min_requerimientos, max_requerimientos) for _ in range(cant_disponibilidad) ]
            
        matrix.append(np.array(req))
    return np.array(matrix)

In [4]:
cant_variables = 5

min_utilidad = 10000
max_utilidad = 20000

In [5]:
utilidades = np.array([randint(min_utilidad, max_utilidad) for _ in range(cant_variables)])
utilidades

array([10045, 10124, 19716, 13007, 10520])

In [6]:
cant_disponibilidad = 5

min_disponibilidad = 100
max_disponibilidad = 500

In [7]:
disponibilidades = np.array([randint(min_disponibilidad, max_disponibilidad) for _ in range(cant_disponibilidad)])
disponibilidades

array([415, 387, 420, 413, 267])

In [8]:
min_requerimientos = 0.0
max_requerimientos = 1.0

In [9]:
tech_matrix = generate_tech_matrix(cant_variables, cant_disponibilidad)
tech_matrix

array([[0.37, 0.55, 0.84, 0.02, 0.62],
       [0.75, 0.26, 0.38, 0.8 , 0.4 ],
       [0.27, 0.85, 0.09, 0.61, 0.07],
       [0.4 , 0.52, 0.69, 0.59, 0.4 ],
       [0.51, 0.43, 0.07, 0.14, 0.29]])

# Solve

In [10]:
# solver initialization
solver = pywraplp.Solver.CreateSolver('GLOP')
inf = solver.infinity()

In [11]:
#Definición de las variables de decisión
#Variables
x = {}
for i in range(cant_variables):
    x[i] = solver.IntVar(0, inf, f'x{i}')
print(f'Number Variables: {solver.NumVariables()}')

Number Variables: 5


In [12]:
#Constraints
for i in range(cant_disponibilidad):
    cons = solver.Sum(tech_matrix[j][i] * x[j] for j in range(cant_variables))
    solver.Add(cons <= disponibilidades[i] )
print(f'Number of constraints: {solver.NumConstraints()}')

Number of constraints: 5


In [13]:
# Objective Function
z = solver.Sum(utilidades[i]*x[i] for i in range(cant_variables))
solver.Maximize(z)

In [1]:
%%time
# solve
status = solver.Solve()

# check result
if status == pywraplp.Solver.OPTIMAL:
    print(solver.Objective().Value())
    for  i in range(cant_variables):
        print(f'x{i}: {x[i].solution_value()}')
else:
    print('No se puede')
print('\n')

NameError: name 'solver' is not defined

# Complete Model

In [8]:
# este codigo utiliza es exactamente lo mismo que en las secciones anteriores
# solo que se ejecuta todo de una sola vez y recibe los parametros necesarios 
# mencionados en el video de mezcla

def mezcla(
    cant_variables = 5, # cantidad de variables de decision
    min_utilidad = 10000, # valor minimo de la utilidad
    max_utilidad = 20000, # valor maximo de la utilidad
    cant_disponibilidad = 5, # cantidad de elementos que restringen la disponibilidad. columnas en la matriz tecnológica.
    min_disponibilidad = 100, # valor minimo de la disponibilidad
    max_disponibilidad = 500, # valor maximo de la disponibilidad
    min_requerimientos = 0.0, # valor minimo que se considerará al crear la matriz tecnológica
    max_requerimientos = 1.0, # valor maximo que se considerará al crear la matriz tecnológica
    get_all_data = False, # si es falso, solamente retorna si es factible o no el problema, el valor optimo y los valores de las variables.
                            # si es verdadero, retorna el tiempo usado, la matriz de utilidad, la matriz de disponibilidad y la matriz tecnológica
    rand_type = 'float' # determina si los valores de la matriz tecnológica son ints o floats
):
    
    
    #gen data
    utilidades = np.array([randint(min_utilidad, max_utilidad) for _ in range(cant_variables)])
    disponibilidades = np.array([randint(min_disponibilidad, max_disponibilidad) for _ in range(cant_disponibilidad)])
    tech_matrix = generate_tech_matrix(cant_variables, cant_disponibilidad, rand_type = rand_type)
    
    #solve
    solver = pywraplp.Solver.CreateSolver('GLOP')
    inf = solver.infinity()
    x = {}
    for i in range(cant_variables):
        x[i] = solver.IntVar(0, inf, f'x{i}')
        
    for i in range(cant_disponibilidad):
        cons = solver.Sum(tech_matrix[j][i] * x[j] for j in range(cant_variables))
        solver.Add(cons <= disponibilidades[i] )
    z = solver.Sum(utilidades[i]*x[i] for i in range(cant_variables))
    solver.Maximize(z)
    
    t_init = time()
    status = solver.Solve()
    t_end = time()
    
    # check result
    result = {}
    if status == pywraplp.Solver.OPTIMAL:
        result['status'] = 'factible'
        result['value'] = solver.Objective().Value()
        result['variable_values()'] = [x[i].solution_value() for i in range(cant_variables)]
        if get_all_data:
            result['time'] = t_end - t_init
            result['utilidades'] = utilidades
            result['disponibilidades'] = disponibilidades
            result['tech_matrix'] = tech_matrix
    else:
        result['status'] = 'infactible'
    
    return result

In [9]:
mezcla(
    cant_variables = 10,
    min_utilidad = 100000,
    max_utilidad = 200000,
    cant_disponibilidad = 3,
    min_disponibilidad = 3,
    max_disponibilidad = 11,
    min_requerimientos = 0,
    max_requerimientos = 2,
    get_all_data = True,
    rand_type = 'float'
)

{'status': 'factible',
 'value': 1785180.8823529412,
 'variable_values()': [4.411764705882353,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.147058823529412,
  0.0,
  0.0,
  0.0],
 'time': 0.03698539733886719,
 'utilidades': array([190182, 157242, 124017, 132115, 187303, 195014, 183822, 174854,
        185704, 107581]),
 'disponibilidades': array([7, 3, 4]),
 'tech_matrix': array([[0.47, 0.68, 0.02],
        [0.13, 0.27, 0.96],
        [0.53, 0.28, 0.72],
        [0.73, 0.44, 0.33],
        [0.51, 0.79, 0.34],
        [0.86, 0.81, 0.58],
        [0.73, 0.  , 0.76],
        [0.12, 0.33, 0.92],
        [0.54, 0.46, 0.98],
        [0.67, 0.87, 0.66]])}